In [1]:
import pandas as pd
import os
import json

In [2]:
df10 = pd.read_csv("Ref10_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df11 = pd.read_csv("Ref11_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df12 = pd.read_csv("Ref12_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df13 = pd.read_csv("Ref13_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df14 = pd.read_csv("Ref14_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )

In [3]:
files = [df10, df11, df12, df13, df14]

for df in files:
    df['count'] = 1

In [4]:
df10_grouped = df10.groupby(['PRMSIC', 'PRMSICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

In [5]:
grouped_files = [df10_grouped, df11_grouped, df12_grouped, df13_grouped, df14_grouped]

In [6]:
cols = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

In [7]:
def getNAICS(row):
    code = str(row['PNACODE'])[0:2]
    return code

In [8]:
def getNAICSLabel(row):
    label = naicsCODES[row['NAICS']] 
    return label

In [9]:
naicsCODES = \
{'11': 'AGRICULTURE, FORESTRY, FISHING AND HUNTING',
 '21': 'MINING',
 '22': 'UTILITIES',
 '23': 'CONSTRUCTION',
 '31': 'MANUFACTURING',
 '32': 'MANUFACTURING',
 '33': 'MANUFACTURING',
 '42': 'WHOLESALE TRADE',
 '44': 'RETAIL TRADE',
 '45': 'RETAIL TRADE',
 '48': 'TRANSPORTATION AND WAREHOUSING',
 '49': 'TRANSPORTATION AND WAREHOUSING',
 '51': 'INFORMATION',
 '52': 'FINANCE AND INSURANCE',
 '53': 'REAL ESTATE RENTAL AND LEASING',
 '54': 'PROFESSIONAL, SCIENTIFIC, AND TECHNICAL SERVICES',
 '55': 'MANAGEMENT OF COMPANIES AND ENTERPRISES',
 '56': 'ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIATION SERVICES',
 '61': 'EDUCATIONAL SERVICES',
 '62': 'HEALTH CARE AND SOCIAL ASSISTANCE',
 '71': 'ARTS, ENTERTAINMENT, AND RECREATION',
 '72': 'ACCOMMODATION AND FOOD SERVICES',
 '81': 'OTHER SERVICES (EXCEPT PUBLIC ADMINISTRATION)',
 '92': 'PUBLIC ADMINISTRATION',
 '99': 'UNCLASSIFIED'}

In [10]:
for df in grouped_files:
    df.columns = df.columns.droplevel(1)
    df.columns = cols 
    del df['count_MED']
    df.loc[:,'NAICS'] = df.apply(getNAICS, axis = 1)
    df.loc[:,'NAICS_LABEL'] = df.apply(getNAICSLabel, axis = 1)

####don't use below code - no need to save

In [ ]:
df10_grouped.to_csv("df10_grouped.csv")
df11_grouped.to_csv("df11_grouped.csv")
df12_grouped.to_csv("df12_grouped.csv") 
df13_grouped.to_csv("df13_grouped.csv")
df14_grouped.to_csv("df14_grouped.csv") 

In [80]:
import geopandas as gp

In [13]:
boros = gp.GeoDataFrame.from_file("../nycem_shapfiles/nyct2010_15b/nyct2010.shp")

####test drive

In [14]:
df = df10_grouped[df10_grouped['SICD'] == 'BEAUTY SALONS']

In [15]:
result = pd.merge(df, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
del result['BoroName']
del result['CT2010']

####Figure out 

In [517]:
import shapefile
from pyproj import Proj, transform 
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon

def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x2,y2 = transform(inProj,outProj,x0,y0)
    
    return (x2, y2)

In [28]:
sf = shapefile.Reader("../nycem_shapfiles/nyct2010_15b/nyct2010.shp")
shapes = sf.shapes()

In [274]:
type(Polygon(convertPolygon(shapes[0].points)))

shapely.geometry.polygon.Polygon

In [269]:
print type(boros.ix[0]['geometry'])

<class 'shapely.geometry.polygon.Polygon'>


In [278]:
>>> p1 = Polygon([(0, 0), (1, 0), (1, 1)])
>>> p2 = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])
>>> p3 = Polygon([(2, 0), (3, 0), (3, 1), (2, 1)])

g = gp.GeoSeries([p1, p2, p3])

In [281]:
print g[0]

POLYGON ((0 0, 1 0, 1 1, 0 0))


In [271]:
def convertPolygon(list_of_points):
    polygon_points = [] 
    for i in list_of_points:
        new_point = tuple(convert(i))
        polygon_points.append(new_point)
    return polygon_points

In [ ]:
boros.apply()

In [208]:
boros['geometry'][0].exterior.coords.xy[0]

array('d', [962269.1260375977, 962288.7224121094, 962548.4429931641, 962368.407409668, 962291.7900390625, 962144.5823974609, 961894.1400146484, 961637.3491821289, 961685.0524291992, 961742.125793457, 961516.8474121094, 961508.2352294922, 961479.7280273438, 961414.4558105469, 960650.7994384766, 960047.7131958008, 960079.5504150391, 960099.8128051758, 960153.1690063477, 960195.0620117188, 960237.9426269531, 960452.7714233398, 960669.7493896484, 960898.8204345703, 961473.7766113281, 961391.0897827148, 961970.9147949219, 962269.1260375977])

####Create Directories with Files

In [100]:
df = grouped_files[4]
year = YEARS[4] 

base_year_loc = base_loc + "/" + year
year_dir = os.listdir(base_year_loc)

In [101]:
YEARS = ["2010", "2011", "2012", "2013", "2014"]
NAICS = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(0))
PNATITL = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(1))
SICD = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(2))

base_loc = 
cnt = len(PNATITL)

In [102]:
def createGeoJSON(pnatitl_loc, string2):
    
    #pnatitl_loc = base_year_loc + "/" + string1
    sicd_code = string2.replace("/", "-") 
    
    df_temp = df[df['SICD'] == string2]
    
    result = pd.merge(df_temp, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
    
    loc = pnatitl_loc + "/" + sicd_code + ".json"
    
    with open(loc, 'wb') as fp:
        json.dump(gp.GeoDataFrame(result).to_json(), fp)

#gp.GeoDataFrame(result).to_json()

In [103]:
for index in range(cnt): 
    
    year_dir = os.listdir(base_year_loc)
    
    naics = NAICS[index]
    pnatitl = PNATITL[index].replace("/", "-")
    sicd = SICD[index]
    
    if naics not in os.listdir(base_year_loc): 
        naics_loc = base_year_loc + "/" + naics
        os.mkdir(naics_loc)
        
        pnatitl_loc = naics_loc + "/" + pnatitl 
        os.mkdir(pnatitl_loc)
        #create file 
        createGeoJSON(pnatitl_loc, sicd)
        
    else: 
        naics_loc = base_year_loc + "/" + naics
        
        if pnatitl not in os.listdir(naics_loc):
            pnatitl_loc = naics_loc + "/" + pnatitl 
            os.mkdir(pnatitl_loc)
            #create file 
            createGeoJSON(pnatitl_loc, sicd)
            
        else: 
            
            pnatitl_loc = naics_loc + "/" + pnatitl 
            #create file 
            createGeoJSON(pnatitl_loc, sicd)     

In [70]:
for year in YEARS:
    year_folder = str(year)
    
    if year_folder in os.listdir(base_dir):
        print 'already created'
    else:
        add_year = base_dir + "/" + year_folder
        os.mkdir(add_year)

NameError: name 'base_dir' is not defined

In [92]:
apm = base_year_loc + "/" + "ABRASIVE PROD MANUFACTURING"
ac = base_year_loc + "/"  + "AC REFRIGERATION & FORCED AIR HEATING"

os.mkdir(apm)
os.mkdir(ac)

In [136]:
# df10_grouped.groupby(['PNATITL', 'SICD']).agg('count').head(10)

x = "INTERNET PUBLISHING/BROADCASTING/WEB SEARCH PRTLS"

In [138]:
x.replace("/", "-")

'INTERNET PUBLISHING-BROADCASTING-WEB SEARCH PRTLS'

In [246]:
convert([1004958.0781860352, 237310.06280517578])

[-73.92519212761768, 40.81801137088798]

In [546]:
result = df10_grouped[df10_grouped['PNATITL'] == 'DIRECT MAIL ADVERTISING']
result1 = pd.merge(result, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])

In [535]:
#testdf = result1[result1["SHP_BOROUGH"] == 'Manhattan'].ix[50:52,]
testdf = result1
# type(testdf['geometry'][0])
#type(testdf['geometry'][2])

In [511]:
x0 = list(testdf['geometry'][2][0].exterior.coords)
x1 = list(testdf['geometry'][2][1].exterior.coords) 

new_list0 = []
new_list1 = []

for i in x0:
    new_list0.append(convert(i))
    
for j in x1:
    new_list1.append(convert(j))
    
x = MultiPolygon((Polygon(new_list0), Polygon(new_list1)))

In [537]:
def testfunction(row):
    geo = row['geometry']
    
    try: 
        x = list(geo.exterior.coords)
        new_list = [] 
    
        for i in x:
            new_list.append(convert(i))
        
        return Polygon(new_list)
    
    except: 
        x0 = list(testdf['geometry'][2][0].exterior.coords)
        x1 = list(testdf['geometry'][2][1].exterior.coords) 

        new_list0 = []
        new_list1 = []

        for i in x0:
            new_list0.append(convert(i))
    
        for j in x1:
            new_list1.append(convert(j))
    
        return MultiPolygon((Polygon(new_list0), Polygon(new_list1)))

In [550]:
result1.loc[:,('geometry')] = result1.apply(testfunction, axis = 1)

In [551]:
testdf = result1.ix[0:3,]

In [572]:
with open('test.geojson', 'wb') as fp:
    json.dump(ast.literal_eval(stringVersion), fp)

In [562]:
import ast
stringVersion= gp.GeoDataFrame(testdf).to_json()

In [566]:

# with open('/Users/krluna/Ref_LMI/test.geojson') as fp:
#     json.dump(ast.literal_eval(stringVersion), fp)

NameError: name 'ls' is not defined

In [571]:
rm test.json

In [380]:
import json
import csv 
import StringIO 

In [381]:
f = open("test.geojson")

In [372]:
x = StringIO.StringIO(f)

In [373]:
print x.getvalue()

<open file 'test.geojson', mode 'r' at 0x11edca030>


In [382]:
testjson = json.load(f)

In [389]:
testjson = testjson.replace(" ", "")

In [545]:
with open("test1.geojson", 'wb') as fp:
        json.dump(testjson, fp)

In [433]:
# testjson